# Tanscripting Video Files From TikTok

#### Installing dependencies
We need moviepy to convert the videos to mp3
We need openai-whisper to tanscript the files to text

In [ ]:
%pip install moviepy
%pip install -U openai-whisper
%pip install faster-whisper 
%pip install pandas 
%pip install requests
%pip install moviepy
%pip install -U spacy
%pip install langdetect
%pip install -U yt-dlp
!brew install ffmpeg
pip install gradio_client
%pip install gradio

In [2]:
%pip install pickle

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle
Note: you may need to restart the kernel to use updated packages.


Set your notebook to the right directrory

In [8]:

%cd "/Users/noursafadi/Documents/Uni/Parsons-Spring-25/MajorStudio02/Thesis/tiktok-scraper"

/Users/noursafadi/Documents/Uni/Parsons-Spring-25/MajorStudio02/Thesis/tiktok-scraper


/Users/noursafadi/.pyenv/versions/3.10.12/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Importing

In [10]:
from moviepy import *
import whisper
import csv
import requests
from faster_whisper import WhisperModel
import httpx
import json
from os import listdir, path
import pandas as pd
import spacy
import yt_dlp

In [22]:
import pickle

### Data

In [11]:
# Load the JSON file into a pandas DataFrame
JSON_PATH = "data/dataset_youtube_vids.json"
with open(JSON_PATH, "r") as file: 
    df = json.load(file)

### Structure

In [ ]:
##index each row in the original dataset 
##index each downloaded audio

#Step 01 
# TODO: Loop throug the dataset 
# TODO: add a new column index starting 0 for each row 

# TODO: Loop through the dataset 
    # TODO: Get URL for each video
    # TODO: Get index 
    # TODO: name the audio index-video

#Step 02
# TODO: Whisper transcript Loop throug the audio files 
# TODO: Extraxt the index name 
# TODO: Match it with index in the original DF
# TODO: Append the transcript to a new column witht

In [12]:
# TODO: Loop throug the dataset 
# TODO: add a new column index starting 0 for each row 
for  idx, row in enumerate(df):
    row["index"] = idx

In [13]:
df_copy = df.copy()

#### YouTube to Audio

In [ ]:
# TODO: Loop through the dataset 
for row in df:
    # TODO: Get URL for each video
    print(row["url"])
    url = row["url"]
    # TODO: Get index 
    print(row["index"])
    id = row["index"]
    # TODO: name the audio index-video
    ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
    'outtmpl': f'audio_files/{id}-audio.%(ext)s'
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

In [79]:
# Load the latest checkpoint
try:
    with open('transcribed_data.pkl', 'rb') as f:
        df_copy = pickle.load(f)
    transcribed_count = sum(1 for item in df_copy if 'transcript' in item and item['transcript'])
    print(f"Loaded checkpoint with {transcribed_count} transcribed files")
except FileNotFoundError:
    print("No checkpoint found. Starting fresh.")
    df_copy = df.copy()  # This requires df to be defined first

Loaded checkpoint with 118 transcribed files


### Using gradio_client hosted on Kaggle to use GPU to run Whisper AI 

In [28]:
from gradio_client import Client
from gradio_client.utils import handle_file

In [74]:
client = Client("https://c79944888f1ae46d67.gradio.live/")

audio_folder = "audio_files/"
audio_files = [f for f in listdir(audio_folder) if f.endswith(".mp3")]

# Track progress - now using the full length
total_files = len(audio_files)
processed = 0
skipped = 0

# Loop through ALL audio files (removed the [:100] slice)
for fname in audio_files:
    file_path = path.join(audio_folder, fname)
    number = int(fname.split("-")[0])
    
    # Find matching item in df_copy
    matching_item = next((item for item in df_copy if item["index"] == number), None)
    
    # Skip if already transcribed
    if matching_item and "transcript" in matching_item and matching_item["transcript"]:
        print(f"Skipping {fname} - already transcribed")
        skipped += 1
        continue
        
    if matching_item:
        print(f"Processing {fname} ({processed+1}/{total_files})")
        try:
            result = client.predict(
                audio=handle_file(file_path),
                api_name="/predict"
            )
            matching_item["transcript"] = result
            print(f"Transcribed: {result[:100]}...")  # Print start of transcript
            
            # Save checkpoint every 10 files (or adjust as needed for larger datasets)
            processed += 1
            if processed % 10 == 0:
                with open('transcribed_data.pkl', 'wb') as f:
                    pickle.dump(df_copy, f)
                print(f"Checkpoint saved ({processed}/{total_files})")
                
        except Exception as e:
            print(f"Error processing {fname}: {e}")
            # Save on error to prevent losing progress
            with open('transcribed_data_error.pkl', 'wb') as f:
                pickle.dump(df_copy, f)
            print("Progress saved after error")
    else:
        print(f"No matching index found for {fname}")

print(f"Finished processing. Transcribed: {processed}, Skipped: {skipped}")

# Save final results
with open('transcribed_data.pkl', 'wb') as f:
    pickle.dump(df_copy, f)

with open('transcribed_data.json', 'w') as f:
    json.dump(df_copy, f, indent=2)

Loaded as API: https://c79944888f1ae46d67.gradio.live/ ✔


ValueError: Could not fetch config for https://c79944888f1ae46d67.gradio.live/

In [77]:
with open('transcribed_data.pkl', 'wb') as f:
    pickle.dump(df_copy, f)

with open('transcribed_data.json', 'w') as f:
    json.dump(df_copy, f, indent=2)